In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import wrangle
import model


from importlib import reload
from itertools import product
import sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [6]:
reload(wrangle)

<module 'wrangle' from '/Users/sinao/coding_projects/oct_2022_unemployment_project/wrangle.py'>

In [7]:
train, X_train, y_train, X_validate, y_validate, X_test, y_test = wrangle.model_prep()

In [8]:
# formulate baseline prediction
train.employed.value_counts(normalize=True)

1    0.968006
0    0.031994
Name: employed, dtype: float64

In [9]:
train   

,household_num,own_bus_or_farm,metro_area_size,age,is_male,veteran,hispanic_or_non,employed,enrolled_in_school,num_children,...,"occupation_Management, business, and financial occupations",occupation_Office and administrative support occupations,occupation_Production occupations,occupation_Professional and related occupations,occupation_Sales and related occupations,occupation_Service occupations,occupation_Transportation and material moving occupations,service_era_n/a,service_era_post_gwot,service_era_pre_gwot
3549,0.000000,0,5.0,37.0,0,0,1,1,0,0.000,...,0,1,0,0,0,0,0,1,0,0
33977,0.083333,0,6.0,35.0,0,0,0,1,0,0.000,...,0,0,0,1,0,0,0,1,0,0
28289,0.166667,1,7.0,43.0,0,0,1,0,0,0.125,...,0,0,0,1,0,0,0,1,0,0
7071,0.333333,0,7.0,58.0,1,0,0,1,0,0.375,...,0,0,0,0,1,0,0,1,0,0
25033,0.166667,0,3.0,39.0,0,0,0,1,0,0.000,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2663,0.083333,1,3.0,46.0,0,0,0,1,0,0.000,...,0,0,0,1,0,0,0,1,0,0
35627,0.250000,0,0.0,21.0,0,0,0,1,1,0.000,...,0,0,0,0,0,1,0,1,0,0
9382,0.166667,0,7.0,45.0,0,0,0,1,0,0.125,...,0,0,0,1,0,0,0,1,0,0
20950,0.083333,0,4.0,27.0,0,0,0,1,0,0.000,...,0,0,0,0,0,1,0,1,0,0


# GridsearchCV

## Decision Tree Classifer

In [10]:
params = {'max_depth': [10,11,12,13,14,15],
          'max_features': [10,20,25,30,35,40]}

dtc = DecisionTreeClassifier()

grid = GridSearchCV(dtc, params, cv=3, scoring='precision')

grid.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [10, 11, 12, 13, 14, 15],
                         'max_features': [10, 20, 25, 30, 35, 40]},
             scoring='precision')

In [11]:
results = grid.cv_results_
results.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'param_max_features', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [12]:
test_scores = results['mean_test_score']

In [13]:
params = results['params']

In [14]:
for p,s in zip(params, test_scores):
    p['score'] = s

pd.DataFrame(params).sort_values(by='score')

,max_depth,max_features,score
0,10,10,0.968013
6,11,10,0.968263
12,12,10,0.968291
14,12,25,0.968346
11,11,40,0.968446
1,10,20,0.968464
30,15,10,0.968464
24,14,10,0.968482
28,14,35,0.968483
7,11,20,0.968497


## K Neighbors

In [15]:
params = {'n_neighbors': [5,6,7,8],
          'weights': ('uniform', 'distance')}

knn = KNeighborsClassifier()

grid = GridSearchCV(knn, params, cv=3, scoring='precision')

grid.fit(X_train, y_train)

/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be acce

GridSearchCV(cv=3, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [5, 6, 7, 8],
                         'weights': ('uniform', 'distance')},
             scoring='precision')

In [16]:
results = grid.cv_results_
results.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_n_neighbors', 'param_weights', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [17]:
test_scores = results['mean_test_score']

In [18]:
params = results['params']

In [19]:
for p,s in zip(params, test_scores):
    p['score'] = s

pd.DataFrame(params).sort_values(by='score', ascending=False)

,n_neighbors,weights,score
3,6,distance,0.968398
2,6,uniform,0.968383
1,5,distance,0.968368
0,5,uniform,0.968300
5,7,distance,0.968239
7,8,distance,0.968170
6,8,uniform,0.968132
4,7,uniform,0.968102


## Logistic Regression

In [20]:
params = {'solver': ('sag', 'saga'),
          'penalty': ['l2'],
          'C': [.1,.5,1.0]}

logr = LogisticRegression()

grid = GridSearchCV(logr, params, cv=3, scoring='precision')

grid.fit(X_train, y_train)

/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/linea

GridSearchCV(cv=3, estimator=LogisticRegression(),
             param_grid={'C': [0.1, 0.5, 1.0], 'penalty': ['l2'],
                         'solver': ('sag', 'saga')},
             scoring='precision')

In [21]:
results = grid.cv_results_
results.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_C', 'param_penalty', 'param_solver', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [22]:
test_scores = results['mean_test_score']

In [23]:
params = results['params']

In [24]:
for p,s in zip(params, test_scores):
    p['score'] = s

pd.DataFrame(params).sort_values(by='score')

,C,penalty,solver,score
0,0.1,l2,sag,0.968006
1,0.1,l2,saga,0.968006
2,0.5,l2,sag,0.968006
3,0.5,l2,saga,0.968006
4,1.0,l2,sag,0.968006
5,1.0,l2,saga,0.968006


In [25]:
from sklearn.svm import LinearSVC

## Linear SVC

In [26]:
params = {'dual':(True, False),
          'C': [1e-5, 1e-4, 1e-3,1,10],
          'max_iter': (500,1000)}

lsvc = LinearSVC()

grid = GridSearchCV(lsvc, params, cv=3, scoring='accuracy')

grid.fit(X_train, y_train)

/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarni

GridSearchCV(cv=3, estimator=LinearSVC(),
             param_grid={'C': [1e-05, 0.0001, 0.001, 1, 10],
                         'dual': (True, False), 'max_iter': (500, 1000)},
             scoring='accuracy')

In [27]:
results = grid.cv_results_
results.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_C', 'param_dual', 'param_max_iter', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [28]:
test_scores = results['mean_test_score']

In [29]:
params = results['params']

In [30]:
for p,s in zip(params, test_scores):
    p['score'] = s

pd.DataFrame(params).sort_values(by='score')

,C,dual,max_iter,score
0,0.00001,True,500,0.968006
17,10.00000,True,1000,0.968006
16,10.00000,True,500,0.968006
15,1.00000,False,1000,0.968006
14,1.00000,False,500,0.968006
13,1.00000,True,1000,0.968006
12,1.00000,True,500,0.968006
11,0.00100,False,1000,0.968006
10,0.00100,False,500,0.968006
9,0.00100,True,1000,0.968006


## Takeaways
* Decision Tree Classifier best performing type of model. Will roll with that through remaining portion of model section



# Modeling with DTC

In [31]:
subsets = [train, X_train, y_train, X_validate, y_validate, X_test, y_test]

In [32]:
feat_set = X_train.columns

In [33]:
def model_dtc(feat_set,\
        model_descriptions,
        comparison_chart,
        subsets):
    
    train=subsets[0]
    X_train=subsets[1]
    y_train=subsets[2]

    features = []
    for feature in feat_set:
        features += [col for col in train.columns if feature in col]

    depths = list(np.arange(20,31,1))
    min_sample_splits = list(np.arange(10,21,1))
    max_features_set = [75]
    selectors = list(product(depths, min_sample_splits, max_features_set))

    for idx, item in enumerate(selectors):
        model_id = 'DTC_'+f'{idx}'
        dtc = DecisionTreeClassifier(max_depth=item[0], min_samples_split=item[1], max_features=item[2],\
                                            random_state=514)
        
        dtc.fit(X_train[features], y_train)

        comparison_chart[model_id] = model.compute_metrics(dtc, X_train[features], y_train).values

        score = dtc.score(X_train[features], y_train).round(4)

        description = pd.DataFrame({'Model': model_id,
                                    'Accuracy(Score)': score,
                                    'Type': 'Decision Tree Classifier',
                                    'Features Used': f'Implicit',
                                    'Parameters': f'Depth: {item[0]},M.Sample Split: {item[1]}, M.Feat: {item[2]}'},
                                    index=[0])

        model_descriptions = pd.concat([model_descriptions, description], ignore_index=True)
        comparison_chart = comparison_chart.copy() #to avoid fragmentation

    return model_descriptions, comparison_chart

In [34]:
comparison_chart = model.create_comp_chart()

In [35]:
model_descriptions = model.create_description_chart(y_train)

In [36]:
model_descriptions, comparison_chart =  model_dtc(feat_set, model_descriptions, comparison_chart, subsets)

In [37]:
comparison_chart.T.sort_values(by='True Negative Rate', ascending=False).head(10)

,Accuracy/Score,True Positives,False Positives,True Negatives,False Negatives,TPR/Recall,False Positive Rate,True Negative Rate,False Negative Rate,Precision,F1-Score,Support Positive,Support Negative
DTC_55,0.9779,26701.0,389.0,501.0,227.0,0.9916,0.4371,0.5629,0.0084,0.9856,0.9886,26928.0,890.0
DTC_66,0.9782,26715.0,393.0,497.0,213.0,0.9921,0.4416,0.5584,0.0079,0.9855,0.9888,26928.0,890.0
DTC_110,0.9772,26698.0,403.0,487.0,230.0,0.9915,0.4528,0.5472,0.0085,0.9851,0.9883,26928.0,890.0
DTC_77,0.9774,26718.0,420.0,470.0,210.0,0.9922,0.4719,0.5281,0.0078,0.9845,0.9883,26928.0,890.0
DTC_88,0.9769,26709.0,424.0,466.0,219.0,0.9919,0.4764,0.5236,0.0081,0.9844,0.9881,26928.0,890.0
DTC_99,0.9774,26724.0,425.0,465.0,204.0,0.9924,0.4775,0.5225,0.0076,0.9843,0.9884,26928.0,890.0
DTC_100,0.9780,26745.0,428.0,462.0,183.0,0.9932,0.4809,0.5191,0.0068,0.9842,0.9887,26928.0,890.0
DTC_44,0.9767,26717.0,436.0,454.0,211.0,0.9922,0.4899,0.5101,0.0078,0.9839,0.9880,26928.0,890.0
DTC_67,0.9773,26734.0,438.0,452.0,194.0,0.9928,0.4921,0.5079,0.0072,0.9839,0.9883,26928.0,890.0
DTC_78,0.9776,26742.0,438.0,452.0,186.0,0.9931,0.4921,0.5079,0.0069,0.9839,0.9885,26928.0,890.0


In [38]:
testy = comparison_chart.T.sort_values(by='True Negative Rate', ascending=False).head(10).index.values.tolist()

In [39]:
top_10 = model_descriptions[model_descriptions.Model.isin(testy)]

In [40]:
model_descriptions.sort_values(by='Accuracy(Score)', ascending=False).head(10)

,Model,Accuracy(Score),Type,Features Used,Parameters
67,DTC_66,0.9782,Decision Tree Classifier,Implicit,"Depth: 26,M.Sample Split: 10, M.Feat: 75"
101,DTC_100,0.9780,Decision Tree Classifier,Implicit,"Depth: 29,M.Sample Split: 11, M.Feat: 75"
56,DTC_55,0.9779,Decision Tree Classifier,Implicit,"Depth: 25,M.Sample Split: 10, M.Feat: 75"
79,DTC_78,0.9776,Decision Tree Classifier,Implicit,"Depth: 27,M.Sample Split: 11, M.Feat: 75"
78,DTC_77,0.9774,Decision Tree Classifier,Implicit,"Depth: 27,M.Sample Split: 10, M.Feat: 75"
100,DTC_99,0.9774,Decision Tree Classifier,Implicit,"Depth: 29,M.Sample Split: 10, M.Feat: 75"
80,DTC_79,0.9774,Decision Tree Classifier,Implicit,"Depth: 27,M.Sample Split: 12, M.Feat: 75"
68,DTC_67,0.9773,Decision Tree Classifier,Implicit,"Depth: 26,M.Sample Split: 11, M.Feat: 75"
112,DTC_111,0.9773,Decision Tree Classifier,Implicit,"Depth: 30,M.Sample Split: 11, M.Feat: 75"
111,DTC_110,0.9772,Decision Tree Classifier,Implicit,"Depth: 30,M.Sample Split: 10, M.Feat: 75"


## Takeaways 
* Round 1
    * Better True Negative Rate performance with higher Depth and lower sample splits. 
    * Max Features between 25 and above performed well
* Round 2
    * Min Sample Split of 10 is consitent high performer. Will go for 10,11,12 as locked in options
    * Higher Max features is helping to drive up True Negative rate will push 50+ then lock in
    * Depth aabove 20 is superior will push 25 plus then lock in
* Round 3
    * added None as max feature option and it outperformed all Min sample split still best at 10
    * Depth still with room for upwards growth
    .9799 Accuracy and 61% TNR with Depth: 30 ,M.Sample Split: 10, M.Feat: None
* Last Round 
    * Lowered Sample Split down to 5 and got better performance
    * Max Features
    * Depth 31 was the best performing with TNR of 87.3% 
    * Will take Top 10 to validate
* Revisiting
    * Use of None for max features is causing overfitting. will switch to range 40~50
* Final Round 
    * Top 10 

In [41]:
for item in top_10.index:
    print(top_10.loc[item]['Parameters'])

Depth: 24,M.Sample Split: 10, M.Feat: 75
Depth: 25,M.Sample Split: 10, M.Feat: 75
Depth: 26,M.Sample Split: 10, M.Feat: 75
Depth: 26,M.Sample Split: 11, M.Feat: 75
Depth: 27,M.Sample Split: 10, M.Feat: 75
Depth: 27,M.Sample Split: 11, M.Feat: 75
Depth: 28,M.Sample Split: 10, M.Feat: 75
Depth: 29,M.Sample Split: 10, M.Feat: 75
Depth: 29,M.Sample Split: 11, M.Feat: 75
Depth: 30,M.Sample Split: 10, M.Feat: 75


In [42]:
pesty = top_10.Parameters.values[0]

In [43]:
top_10.Parameters.values[0]

'Depth: 24,M.Sample Split: 10, M.Feat: 75'

In [44]:
import re

In [45]:
lesty = re.findall(r'\d\d', pesty)

In [46]:
def validate_dtc(feat_set,\
        top_10,
        subsets):
    
    train=subsets[0]
    X_train=subsets[1]
    y_train=subsets[2]
    X_validate=subsets[3]
    y_validate=subsets[4]

    features = []
    for feature in feat_set:
        features += [col for col in train.columns if feature in col]
        
    depths = list(np.arange(20,35,1))
    min_sample_splits = list(np.arange(5,13,1))
    max_features_set = [55, 75, None]
    selectors = list(product(depths, min_sample_splits, max_features_set))


    validate_descriptions = model.create_description_chart(y_train)
    val_comparisons = model.create_comp_chart()

    for item in top_10.index:
        model_id = top_10.loc[item]['Model']
        selectors = re.findall(r'\d\d', top_10.loc[item]['Parameters'])

        dtc = DecisionTreeClassifier(max_depth=int(selectors[0]), min_samples_split=int(selectors[1]), max_features=int(selectors[2]),\
                                            random_state=514)
        
        dtc.fit(X_train[features], y_train)

        val_comparisons[model_id] = model.compute_metrics(dtc, X_validate[features], y_validate).values

        score = dtc.score(X_validate[features], y_validate).round(4)

        description = pd.DataFrame({'Model': model_id,
                                    'Accuracy(Score)': score,
                                    'Type': 'Decision Tree Classifier',
                                    'Features Used': f'{feat_set}',
                                    'Parameters': f'Depth: {selectors[0]}, M.Sample Split: {selectors[1]}, M.Feat: {selectors[2]}'},
                                    index=[0])

        validate_descriptions = pd.concat([validate_descriptions, description], ignore_index=True)
        val_comparisons = val_comparisons.copy()

    return validate_descriptions, val_comparisons

In [47]:
validate_descriptions, val_comparisons =  validate_dtc(feat_set, top_10, subsets)

In [48]:
val_comparisons.T.sort_values(by='True Negative Rate', ascending=False)

,Accuracy/Score,True Positives,False Positives,True Negatives,False Negatives,TPR/Recall,False Positive Rate,True Negative Rate,False Negative Rate,Precision,F1-Score,Support Positive,Support Negative
DTC_55,0.9432,11215.0,351.0,30.0,326.0,0.9718,0.9213,0.0787,0.0282,0.9697,0.9707,11541.0,381.0
DTC_44,0.9459,11248.0,352.0,29.0,293.0,0.9746,0.9239,0.0761,0.0254,0.9697,0.9721,11541.0,381.0
DTC_67,0.9508,11307.0,352.0,29.0,234.0,0.9797,0.9239,0.0761,0.0203,0.9698,0.9747,11541.0,381.0
DTC_99,0.9475,11269.0,354.0,27.0,272.0,0.9764,0.9291,0.0709,0.0236,0.9695,0.9730,11541.0,381.0
DTC_110,0.9454,11246.0,356.0,25.0,295.0,0.9744,0.9344,0.0656,0.0256,0.9693,0.9719,11541.0,381.0
DTC_66,0.9460,11255.0,358.0,23.0,286.0,0.9752,0.9396,0.0604,0.0248,0.9692,0.9722,11541.0,381.0
DTC_100,0.9463,11261.0,360.0,21.0,280.0,0.9757,0.9449,0.0551,0.0243,0.9690,0.9724,11541.0,381.0
DTC_77,0.9442,11238.0,362.0,19.0,303.0,0.9737,0.9501,0.0499,0.0263,0.9688,0.9713,11541.0,381.0
DTC_78,0.9467,11268.0,363.0,18.0,273.0,0.9763,0.9528,0.0472,0.0237,0.9688,0.9726,11541.0,381.0
DTC_88,0.9438,11236.0,365.0,16.0,305.0,0.9736,0.9580,0.0420,0.0264,0.9685,0.9710,11541.0,381.0


## Validate Takeaways
* Round 1
    * Huge performance loss, appears model was over fit. Will adjust 
* Round 2 
    * Pulled in more models but unable to get above 13% TNR. Will redjust starting from Train section
* Round 3
    * Lower depth performing better with larger min sample splits. Need to increase generalisability
* Final Round
    * Unable to crack 12% with adjustments. Will follow on to model in order to finish MVP and revisit different model types 

In [49]:
validate_descriptions.sort_values(by='Accuracy(Score)', ascending=False)

,Model,Accuracy(Score),Type,Features Used,Parameters
4,DTC_67,0.9508,Decision Tree Classifier,"Index(['household_num', 'own_bus_or_farm', 'me...","Depth: 26, M.Sample Split: 11, M.Feat: 75"
8,DTC_99,0.9475,Decision Tree Classifier,"Index(['household_num', 'own_bus_or_farm', 'me...","Depth: 29, M.Sample Split: 10, M.Feat: 75"
6,DTC_78,0.9467,Decision Tree Classifier,"Index(['household_num', 'own_bus_or_farm', 'me...","Depth: 27, M.Sample Split: 11, M.Feat: 75"
9,DTC_100,0.9463,Decision Tree Classifier,"Index(['household_num', 'own_bus_or_farm', 'me...","Depth: 29, M.Sample Split: 11, M.Feat: 75"
3,DTC_66,0.9460,Decision Tree Classifier,"Index(['household_num', 'own_bus_or_farm', 'me...","Depth: 26, M.Sample Split: 10, M.Feat: 75"
1,DTC_44,0.9459,Decision Tree Classifier,"Index(['household_num', 'own_bus_or_farm', 'me...","Depth: 24, M.Sample Split: 10, M.Feat: 75"
10,DTC_110,0.9454,Decision Tree Classifier,"Index(['household_num', 'own_bus_or_farm', 'me...","Depth: 30, M.Sample Split: 10, M.Feat: 75"
5,DTC_77,0.9442,Decision Tree Classifier,"Index(['household_num', 'own_bus_or_farm', 'me...","Depth: 27, M.Sample Split: 10, M.Feat: 75"
7,DTC_88,0.9438,Decision Tree Classifier,"Index(['household_num', 'own_bus_or_farm', 'me...","Depth: 28, M.Sample Split: 10, M.Feat: 75"
2,DTC_55,0.9432,Decision Tree Classifier,"Index(['household_num', 'own_bus_or_farm', 'me...","Depth: 25, M.Sample Split: 10, M.Feat: 75"


In [50]:
top_1 = validate_descriptions.iloc[2]

## Test Subset

In [51]:
def test_dtc(feat_set,\
        top_1,
        subsets):
    
    train=subsets[0]
    X_train=subsets[1]
    y_train=subsets[2]
    X_test=subsets[5]
    y_test=subsets[6]

    features = []
    for feature in feat_set:
        features += [col for col in train.columns if feature in col]

    model_id = top_1['Model']
    selectors = re.findall(r'\d\d', top_1['Parameters'])
    dtc = DecisionTreeClassifier(max_depth=int(selectors[0]), min_samples_split=int(selectors[1]), max_features=int(selectors[2]),\
                                            random_state=514)
    
    dtc.fit(X_train[features], y_train)

    comparison_chart = model.create_comp_chart()
    comparison_chart[model_id] = model.compute_metrics(dtc, X_test[features], y_test).values

    score = dtc.score(X_test[features], y_test).round(4)

    description = pd.DataFrame({'Model': model_id,
                                'Accuracy(Score)': score,
                                'Type': 'Decision Tree Classifier',
                                'Features Used': f'{feat_set}',
                                'Parameters': f'Depth: {selectors[0]}'},
                                index=[0])

    return description, comparison_chart

In [52]:
test_description, test_chart = test_dtc(feat_set, top_1, subsets)

In [53]:
test_chart

,DTC_78
Accuracy/Score,0.9475
True Positives,9394.0000
False Positives,299.0000
True Negatives,19.0000
False Negatives,223.0000
TPR/Recall,0.9768
False Positive Rate,0.9403
True Negative Rate,0.0597
False Negative Rate,0.0232
Precision,0.9692


## Takeaways 
* Even worse performance on with the True Negative rate. Attempting to control for overfitting caused a very strong hit. Will have to readjust next go through. 

# Modeling With Linear SVC

In [54]:
params = {'dual':[False],
          'C': [1e-5, 1e-4, 1e-3,1e-2, .1,.2,1,10],
          'max_iter': (500,1000)}

lsvc = LinearSVC()

grid = GridSearchCV(lsvc, params, cv=3, scoring='accuracy')

grid.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=LinearSVC(),
             param_grid={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 0.2, 1, 10],
                         'dual': [False], 'max_iter': (500, 1000)},
             scoring='accuracy')

In [55]:
results = grid.cv_results_
results.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_C', 'param_dual', 'param_max_iter', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [56]:
test_scores = results['mean_test_score']

In [57]:
params = results['params']

In [58]:
for p,s in zip(params, test_scores):
    p['score'] = s

pd.DataFrame(params).sort_values(by='score')

,C,dual,max_iter,score
0,0.00001,False,500,0.968006
1,0.00001,False,1000,0.968006
2,0.00010,False,500,0.968006
3,0.00010,False,1000,0.968006
4,0.00100,False,500,0.968006
5,0.00100,False,1000,0.968006
6,0.01000,False,500,0.968006
7,0.01000,False,1000,0.968006
8,0.10000,False,500,0.968006
9,0.10000,False,1000,0.968006


In [59]:
def model_lsvc(feat_set,\
        model_descriptions,
        comparison_chart,
        subsets):
    
    train=subsets[0]
    X_train=subsets[1]
    y_train=subsets[2]


    features = []
    for feature in feat_set:
        features += [col for col in train.columns if feature in col]


    C = [.1,.2,1]
    selectors = list(product(C))

    for idx, item in enumerate(selectors):
        model_id = 'LSVC_'+f'{idx}'
        lsvc = LinearSVC(dual=False,  C=item[0])
        
        lsvc.fit(X_train[features], y_train)

        comparison_chart[model_id] = model.compute_metrics(lsvc, X_train[features], y_train).values

        score = lsvc.score(X_train[features], y_train).round(4)

        description = pd.DataFrame({'Model': model_id,
                                    'Accuracy(Score)': score,
                                    'Type': 'Linear SVC',
                                    'Features Used': f'Implicit',
                                    'Parameters': f'C: {item[0]}'},
                                    index=[0])

        model_descriptions = pd.concat([model_descriptions, description], ignore_index=True)
        comparison_chart = comparison_chart.copy() #to avoid fragmentation

    return model_descriptions, comparison_chart

In [60]:
lsvc_chart = model.create_comp_chart()

In [61]:
lsvc_descriptions = model.create_description_chart(y_train)

In [62]:
lsvc_descriptions, lsvc_chart =  model_lsvc(feat_set, lsvc_descriptions, lsvc_chart, subsets)

In [63]:
lsvc_descriptions

,Model,Accuracy(Score),Type,Features Used,Parameters
0,Baseline,0.000,Basic Baseline,Baseline Prediction,n/a
1,LSVC_0,0.968,Linear SVC,Implicit,C: 0.1
2,LSVC_1,0.968,Linear SVC,Implicit,C: 0.2
3,LSVC_2,0.968,Linear SVC,Implicit,C: 1


In [64]:
float(lsvc_descriptions.Parameters[1].replace('C: ',''))

0.1

In [65]:
pesty= lsvc_descriptions.Parameters[1]

In [66]:
re.findall(r'\d.\d', pesty)

['0.1']

In [67]:
lsvc_chart.T

,Accuracy/Score,True Positives,False Positives,True Negatives,False Negatives,TPR/Recall,False Positive Rate,True Negative Rate,False Negative Rate,Precision,F1-Score,Support Positive,Support Negative
LSVC_0,0.968,26928.0,890.0,0.0,0.0,1.0,1.0000,0.0000,0.0,0.968,0.9837,26928.0,890.0
LSVC_1,0.968,26928.0,890.0,0.0,0.0,1.0,1.0000,0.0000,0.0,0.968,0.9837,26928.0,890.0
LSVC_2,0.968,26928.0,889.0,1.0,0.0,1.0,0.9989,0.0011,0.0,0.968,0.9838,26928.0,890.0


## Takeaways
* Approach not useful. On first attempt model did not predict any `unemployed`. Will research model parameters more, and think about reattempting with it. Next will examine other model types/approaches for final round on this iteration of the project. 

# Using XGBoost 

In [68]:
import xgboost as xgb

In [69]:
xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=10, seed=123, learning_rate=1, max_depth=10, eval_metric='auc')

In [70]:

xg_cl.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=1, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=10, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=10, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=123, ...)

In [71]:
preds=xg_cl.predict(X_train)

In [72]:
accuracy=float(np.sum(preds==y_train))/y_train.shape[0]


In [73]:
print(f'accuracy: {accuracy}')

accuracy: 0.9771730534186498


In [74]:
xgb_chart = model.create_comp_chart()

In [75]:
xgb_descriptions = model.create_description_chart(y_train)

In [76]:
xgb_chart['xgb_3'] = model.compute_metrics(xg_cl, X_train, y_train).values

In [77]:
xgb_chart

,xgb_3
Accuracy/Score,0.9772
True Positives,26907.0000
False Positives,614.0000
True Negatives,276.0000
False Negatives,21.0000
TPR/Recall,0.9992
False Positive Rate,0.6899
True Negative Rate,0.3101
False Negative Rate,0.0008
Precision,0.9777


In [78]:
xgb_val_chart = model.create_comp_chart()

In [79]:
xgb_val_descriptions = model.create_description_chart(y_validate)

In [80]:
xgb_val_chart['xgb_3'] = model.compute_metrics(xg_cl, X_validate, y_validate).values

In [81]:
xgb_val_chart

,xgb_3
Accuracy/Score,0.9631
True Positives,11472.0000
False Positives,371.0000
True Negatives,10.0000
False Negatives,69.0000
TPR/Recall,0.9940
False Positive Rate,0.9738
True Negative Rate,0.0262
False Negative Rate,0.0060
Precision,0.9687


## Takeaways
* After three iteractions XGBoost does not appear to offer much greater performance above DTC on the train dataset, and worse performance on the validate. Will need to adjust approach for next go around. 

# Modeling Summary
* Decided to focus on only three types of models for this iteration of project.
* DTC modeling showed promise, but was prone to overfitting when it came time for use on the validation and test subsets
* Linear SVC and XGBOOST performed worse during the model phase, and they were not moved forward.
* Further iterations of project will focus much more on exploration and feature reduction in order to reduce noise and dimensionality. 

# Next Steps
* Will try a SVM with RBF and also take a look at RF algorithms for next iteration of project.